In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `Inference_API_KEY_8B` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.8/978.8 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 20.7 MB/s eta 0:00:00


In [5]:
!pip install opencv-python numpy ultralytics transformers pillow matplotlib seaborn pandas torch reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 15.6 MB/s eta 0:00:00


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from transformers import pipeline, T5ForConditionalGeneration, T5Tokenizer
from PIL import Image
import re
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import torch
import gc
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image as ReportLabImage
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch

# Install dependencies
print("Installing dependencies...")
!pip install opencv-python numpy ultralytics transformers pillow matplotlib seaborn pandas torch reportlab

# Clear any lingering GPU memory (though we're running on CPU)
torch.cuda.empty_cache() if torch.cuda.is_available() else None
gc.collect()

# Verify input files
model_path = r"/content/best.pt"
video_path = r"/content/test_10s.mp4"
if not os.path.exists(model_path):
    print(f"Error: YOLO model file not found at {model_path}")
    exit()
if not os.path.exists(video_path):
    print(f"Error: Video file not found at {video_path}")
    exit()

# Load YOLO model
try:
    model = YOLO(model_path)
    print("YOLO model loaded successfully.")
except Exception as e:
    print(f"Error loading YOLO model: {e}")
    exit()

# Video setup
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print(f"Error: Could not open video at {video_path}")
    exit()
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
middle_index = total_frames // 2 if total_frames > 0 else -1
fps = cap.get(cv2.CAP_PROP_FPS)
selected_frame = None
peak_frame = None
vehicle_counts = {}
track_id_to_class = {}
frame_vehicle_counts = []
frame_index = 0
max_vehicles = 0
max_frame_index = 0
emergency_alerts = []
track_positions = defaultdict(list)
average_speeds = {}
congestion_indices = []

# Process video
print("Processing video...")
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break
    frame = cv2.resize(frame, (700, 500))
    results = model.track(frame, persist=True)
    boxes = results[0].boxes.xyxy.cpu().numpy()
    confidences = results[0].boxes.conf.cpu().numpy()
    classes = results[0].boxes.cls.cpu().numpy().astype(int)
    track_ids = results[0].boxes.id.cpu().numpy() if results[0].boxes.id is not None else []

    # Unique vehicle counts and speed estimation
    current_frame_counts = defaultdict(int)
    for conf, cls, track_id, box in zip(confidences, classes, track_ids, boxes):
        if conf < 0.5:
            continue
        label = results[0].names[cls]
        if track_id not in track_id_to_class:
            track_id_to_class[track_id] = label
            vehicle_counts[label] = vehicle_counts.get(label, 0) + 1
        current_frame_counts[label] += 1
        # Track position for speed
        center_x = (box[0] + box[2]) / 2
        center_y = (box[1] + box[3]) / 2
        track_positions[track_id].append((frame_index, center_x, center_y))

    frame_vehicle_counts.append(dict(current_frame_counts))

    # Emergency vehicle alerts
    if current_frame_counts.get("Ambulance", 0) > 1:
        alert = f"High ambulance activity at {frame_index/fps:.2f}s: {current_frame_counts['Ambulance']} ambulances"
        emergency_alerts.append(alert)
        print(f"Emergency alert: {alert}")

    # Congestion index
    total_in_frame = sum(current_frame_counts.values())
    congestion_index = total_in_frame / 5.0
    congestion_indices.append(congestion_index)
    if total_in_frame > max_vehicles:
        max_vehicles = total_in_frame
        max_frame_index = frame_index
        peak_frame = results[0].plot()

    # Save middle frame
    if frame_index == middle_index:
        selected_frame = results[0].plot()

    frame_index += 1

cap.release()
cv2.destroyAllWindows()
print(f"Video processing complete: {total_frames} frames processed.")

# Calculate average speeds
print("Calculating average speeds...")
for track_id, positions in track_positions.items():
    label = track_id_to_class[track_id]
    if len(positions) < 2:
        continue
    total_speed = 0
    count = 0
    for i in range(1, len(positions)):
        frame_diff = positions[i][0] - positions[i-1][0]
        if frame_diff == 0:
            continue
        dx = positions[i][1] - positions[i-1][1]
        dy = positions[i][2] - positions[i-1][2]
        distance = np.sqrt(dx**2 + dy**2)
        time = frame_diff / fps
        speed = distance / time
        total_speed += speed
        count += 1
    if count > 0:
        avg_speed = total_speed / count
        average_speeds[label] = average_speeds.get(label, 0) + avg_speed
        average_speeds[f"{label}_count"] = average_speeds.get(f"{label}_count", 0) + 1

for label in vehicle_counts.keys():
    count_key = f"{label}_count"
    if count_key in average_speeds:
        average_speeds[label] = average_speeds[label] / average_speeds[count_key]
        del average_speeds[count_key]

# Save annotated frames
try:
    if selected_frame is not None:
        cv2.imwrite("middle_frame.jpg", selected_frame)
        print("Middle frame saved as 'middle_frame.jpg'.")
    else:
        print("Warning: Middle frame not saved.")
    if peak_frame is not None:
        cv2.imwrite("peak_frame.jpg", peak_frame)
        print("Peak frame saved as 'peak_frame.jpg'.")
    else:
        print("Warning: Peak frame not saved.")
except Exception as e:
    print(f"Error saving frames: {e}")

# Convert middle frame to PIL
if selected_frame is not None:
    selected_frame_rgb = cv2.cvtColor(selected_frame, cv2.COLOR_BGR2RGB)
    selected_frame_pil = Image.fromarray(selected_frame_rgb)
else:
    print("Warning: No middle frame selected for PIL conversion.")
    selected_frame_pil = None

# Load report pipeline
print("Loading report pipeline...")
try:
    pipe = pipeline("text2text-generation", model="google/flan-t5-small")
    print("Flan-T5-Small loaded for report generation.")
except Exception as e:
    print(f"Report model error: {e}")
    pipe = None

# Load chatbot model
print("Loading Flan-T5-Large chatbot model...")
try:
    chatbot = pipeline("text2text-generation", model="google/flan-t5-large")
    print("Flan-T5-Large loaded for chatbot.")
except Exception as e:
    print(f"Flan-T5-Large error: {e}")
    print("Chatbot unavailable. Using rule-based answers only.")
    chatbot = None
finally:
    gc.collect()

# Calculate averages and congestion
average_counts = {}
for vtype in vehicle_counts.keys():
    total = sum(frame_counts.get(vtype, 0) for frame_counts in frame_vehicle_counts)
    average_counts[vtype] = total / len(frame_vehicle_counts) if frame_vehicle_counts else 0
max_time_sec = max_frame_index / fps if fps > 0 else 0
average_congestion = np.mean(congestion_indices) if congestion_indices else 0

# Generate traffic density heatmap
print("Generating traffic density heatmap...")
times = [i / fps for i in range(len(frame_vehicle_counts))]
total_vehicles_per_frame = [sum(frame_counts.values()) for frame_counts in frame_vehicle_counts]
plt.figure(figsize=(10, 4))
sns.heatmap([total_vehicles_per_frame], cmap="YlOrRd", xticklabels=50, cbar_kws={'label': 'Vehicle Count'})
plt.xlabel("Time (seconds)")
plt.ylabel("Density")
plt.title("Traffic Density Heatmap")
plt.xticks(ticks=np.linspace(0, len(times)-1, 5), labels=[f"{t:.1f}" for t in np.linspace(0, max(times), 5)])
try:
    plt.savefig("heatmap.png")
    plt.close()
    print("Heatmap saved as 'heatmap.png'.")
except Exception as e:
    print(f"Error saving heatmap: {e}")

# Export to CSV
print("Exporting data to CSV...")
csv_data = {
    "Frame": list(range(len(frame_vehicle_counts))),
    "Time (s)": times,
    "Total Vehicles": total_vehicles_per_frame,
    "Congestion Index": congestion_indices
}
for vtype in vehicle_counts.keys():
    csv_data[vtype] = [frame_counts.get(vtype, 0) for frame_counts in frame_vehicle_counts]
df = pd.DataFrame(csv_data)
try:
    df.to_csv("traffic_data.csv", index=False)
    print("Traffic data exported to 'traffic_data.csv'.")
except Exception as e:
    print(f"Error exporting CSV: {e}")

# Generate text report
print("Generating text report...")
report_prompt = (
    f"Vehicle Detection Report:\n"
    f"Unique vehicle counts across all frames:\n"
)
for vtype, count in vehicle_counts.items():
    report_prompt += f"{vtype}: {count}\n"
report_prompt += "Additional Insights:\n"
report_prompt += "Average vehicles per frame:\n"
for vtype, avg in average_counts.items():
    report_prompt += f"{vtype}: {avg:.2f}\n"
report_prompt += f"Average speeds (pixels/s):\n"
for vtype, speed in average_speeds.items():
    report_prompt += f"{vtype}: {speed:.2f}\n"
report_prompt += f"Peak traffic at {max_time_sec:.2f} seconds with {max_vehicles} vehicles.\n"
report_prompt += f"Average congestion index: {average_congestion:.2f} (0=low, 1=moderate, >2=high).\n"
if emergency_alerts:
    report_prompt += "Emergency Alerts:\n" + "\n".join(emergency_alerts) + "\n"
if frame_vehicle_counts and middle_index < len(frame_vehicle_counts):
    selected_counts = frame_vehicle_counts[middle_index]
    report_prompt += "Selected middle frame shows:\n"
    for vtype, count in selected_counts.items():
        report_prompt += f"{vtype}: {count}\n"
report_prompt += "Generate a detailed summary based on this data and describe the scene in the selected frame."

try:
    if pipe:
        result = pipe(report_prompt, max_new_tokens=300)
        report_text = result[0]['generated_text']
        print("Text report generated with text model.")
    else:
        report_text = "Report failed. Counts:\n" + "\n".join([f"{v}: {c}" for v, c in vehicle_counts.items()])
        print("Warning: Text report generation failed, using fallback.")
except Exception as e:
    print(f"Report error: {e}")
    report_text = "Report failed. Counts:\n" + "\n".join([f"{v}: {c}" for v, c in vehicle_counts.items()])
    print("Warning: Text report generation failed, using fallback.")

print("\n--- Generated Report ---\n")
print(report_text)

# Visualization Dashboard
print("Generating visualization dashboard...")
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(10, 20))
ax1.plot(times, total_vehicles_per_frame, color='blue')
ax1.set_xlabel('Time (seconds)')
ax1.set_ylabel('Total Vehicles')
ax1.set_title('Total Vehicles Over Time')
ax1.grid(True)
vehicle_types = list(vehicle_counts.keys())
for vtype in vehicle_types:
    counts = [frame_counts.get(vtype, 0) for frame_counts in frame_vehicle_counts]
    ax2.plot(times, counts, label=vtype)
ax2.set_xlabel('Time (seconds)')
ax2.set_ylabel('Vehicle Count')
ax2.set_title('Vehicle Counts by Type Over Time')
ax2.legend()
ax2.grid(True)
ax3.bar(vehicle_types, [vehicle_counts[vtype] for vtype in vehicle_types], color='green')
ax3.set_xlabel('Vehicle Type')
ax3.set_ylabel('Unique Count')
ax3.set_title('Unique Vehicle Counts')
ax3.grid(True, axis='y')
ax4.bar(vehicle_types, [average_speeds.get(vtype, 0) for vtype in vehicle_types], color='orange')
ax4.set_xlabel('Vehicle Type')
ax4.set_ylabel('Average Speed (pixels/s)')
ax4.set_title('Average Vehicle Speeds')
ax4.grid(True, axis='y')
plt.tight_layout()
try:
    plt.savefig("dashboard.png")
    plt.close()
    print("Dashboard saved as 'dashboard.png'.")
except Exception as e:
    print(f"Error saving dashboard: {e}")

# Generate PDF report
print("Generating PDF report...")
try:
    pdf = SimpleDocTemplate("report.pdf", pagesize=letter)
    styles = getSampleStyleSheet()
    normal_style = ParagraphStyle(name='NormalWrap', parent=styles['Normal'], wordWrap='CJK')
    heading_style = styles['Heading1']
    subheading_style = styles['Heading2']
    elements = []

    # Title
    elements.append(Paragraph("Vehicle Detection Report", heading_style))
    elements.append(Spacer(1, 0.2 * inch))

    # Summary
    elements.append(Paragraph("Summary", subheading_style))
    elements.append(Spacer(1, 0.1 * inch))
    report_text_lines = report_text.split('\n')
    for line in report_text_lines:
        elements.append(Paragraph(line, normal_style))
    elements.append(Spacer(1, 0.2 * inch))

    # Vehicle Statistics Table
    elements.append(Paragraph("Vehicle Statistics", subheading_style))
    elements.append(Spacer(1, 0.1 * inch))
    table_data = [['Vehicle Type', 'Unique Count', 'Avg/Frame', 'Middle Frame', 'Avg Speed (px/s)']]
    for vtype in vehicle_counts.keys():
        unique_count = vehicle_counts.get(vtype, 0)
        avg_count = average_counts.get(vtype, 0)
        middle_count = frame_vehicle_counts[middle_index].get(vtype, 0) if middle_index < len(frame_vehicle_counts) else 0
        speed = average_speeds.get(vtype, 0)
        table_data.append([vtype, str(unique_count), f"{avg_count:.2f}", str(middle_count), f"{speed:.2f}"])
    table = Table(table_data)
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 10),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
        ('GRID', (0, 0), (-1, -1), 1, colors.black)
    ]))
    elements.append(table)
    elements.append(Spacer(1, 0.2 * inch))

    # Key Insights
    elements.append(Paragraph("Key Insights", subheading_style))
    elements.append(Spacer(1, 0.1 * inch))
    elements.append(Paragraph(f"Peak traffic occurred at {max_time_sec:.2f} seconds with {max_vehicles} vehicles.", normal_style))
    elements.append(Paragraph(f"Average congestion index: {average_congestion:.2f} (0=low, 1=moderate, >2=high).", normal_style))
    if emergency_alerts:
        elements.append(Paragraph("Emergency Alerts:", normal_style))
        for alert in emergency_alerts:
            elements.append(Paragraph(f"- {alert}", normal_style))
    elements.append(Spacer(1, 0.2 * inch))

    # Visualizations
    elements.append(Paragraph("Visualizations", subheading_style))
    elements.append(Spacer(1, 0.1 * inch))
    image_paths = [
        ("dashboard.png", "Visualization Dashboard"),
        ("heatmap.png", "Traffic Density Heatmap"),
        ("middle_frame.jpg", "Middle Frame"),
        ("peak_frame.jpg", "Peak Traffic Frame")
    ]
    for path, title in image_paths:
        if os.path.exists(path):
            try:
                img = ReportLabImage(path, width=5*inch, height=3*inch)
                elements.append(Paragraph(title, normal_style))
                elements.append(img)
                elements.append(Spacer(1, 0.1 * inch))
            except Exception as e:
                print(f"Error adding image {path} to PDF: {e}")
                elements.append(Paragraph(f"{title} not available.", normal_style))
                elements.append(Spacer(1, 0.1 * inch))
        else:
            print(f"Warning: Image {path} not found.")
            elements.append(Paragraph(f"{title} not available.", normal_style))
            elements.append(Spacer(1, 0.1 * inch))

    # Note
    elements.append(Paragraph("Note: This PDF contains the complete vehicle detection report.", normal_style))

    # Build PDF
    pdf.build(elements)
    print("PDF report generated as 'report.pdf'.")
except Exception as e:
    print(f"Error generating PDF: {e}")

# Create optimized context for chatbot
context = (
    f"Video Analysis: {total_frames} frames, {(total_frames/fps):.2f}s, {fps:.2f} FPS.\n"
    f"Unique vehicles: {', '.join([f'{v}: {c}' for v, c in vehicle_counts.items()])} (total: {sum(vehicle_counts.values())}).\n"
    f"Average per frame: {', '.join([f'{v}: {c:.2f}' for v, c in average_counts.items()])}.\n"
    f"Average speeds (pixels/s): {', '.join([f'{v}: {s:.2f}' for v, s in average_speeds.items()])}.\n"
    f"Peak traffic: {max_vehicles} vehicles at {max_time_sec:.2f}s.\n"
    f"Middle frame ({middle_index/fps:.2f}s): {', '.join([f'{v}: {c}' for v, c in frame_vehicle_counts[middle_index].items()]) if middle_index < len(frame_vehicle_counts) else 'no vehicles'}.\n"
    f"Vehicle types: {', '.join(vehicle_counts.keys())}.\n"
    f"Traffic: {'Heavy' if sum(vehicle_counts.values()) > 30 else 'Moderate' if sum(vehicle_counts.values()) > 15 else 'Light'}.\n"
    f"Congestion index: {average_congestion:.2f} (0=low, 1=moderate, >2=high).\n"
)
if emergency_alerts:
    context += f"Emergency alerts: {'; '.join(emergency_alerts)}.\n"
print("\nChatbot context:\n", context)

# Updated answer_question function
def answer_question(question, vehicle_counts, frame_vehicle_counts, fps, total_frames, average_counts, max_time_sec, max_vehicles, context, chatbot, average_speeds, congestion_indices, emergency_alerts):
    question = question.lower().strip()
    if not question:
        return "Please ask a valid question."

    # Rule-based answers
    time_match = re.search(r"at (\d+) seconds", question)
    if time_match:
        time_sec = int(time_match.group(1))
        if time_sec * fps >= total_frames:
            return f"Time {time_sec} seconds exceeds video duration ({total_frames/fps:.2f}s)."
        frame_index = min(int(time_sec * fps), total_frames - 1)
        frame_counts = frame_vehicle_counts[frame_index]
        for vtype in vehicle_counts.keys():
            if vtype.lower() in question:
                count = frame_counts.get(vtype, 0)
                return f"At {time_sec} seconds, there were {count} {vtype}(s)."
        if "traffic" in question or "how many" in question:
            counts_str = ", ".join([f"{count} {vtype}(s)" for vtype, count in frame_counts.items()])
            return f"At {time_sec} seconds: {counts_str or 'no vehicles'}."
        total = sum(frame_counts.values())
        return f"At {time_sec} seconds, there were {total} vehicles."

    if "peak" in question and "time" in question:
        return f"Peak traffic was at {max_time_sec:.2f} seconds with {max_vehicles} vehicles."

    if "average" in question and "speed" in question:
        for vtype in vehicle_counts.keys():
            if vtype.lower() in question:
                speed = average_speeds.get(vtype, 0)
                return f"Average speed of {vtype}s: {speed:.2f} pixels/s."
        speeds_str = ", ".join([f"{vtype}: {speed:.2f} pixels/s" for vtype, speed in average_speeds.items()])
        return f"Average speeds: {speeds_str or 'none'}."

    if "average" in question:
        for vtype in vehicle_counts.keys():
            if vtype.lower() in question:
                avg = average_counts.get(vtype, 0)
                return f"Average {vtype}s per frame: {avg:.2f}."
        averages_str = ", ".join([f"{vtype}: {avg:.2f}" for vtype, avg in average_counts.items()])
        return f"Average vehicles per frame: {averages_str or 'none'}."

    if re.search(r"how many (total |)vehicles", question) or "total number" in question:
        total = sum(vehicle_counts.values())
        return f"Total unique vehicles: {total}."

    for vtype, count in vehicle_counts.items():
        type_lower = vtype.lower()
        if any(phrase in question for phrase in [f"how many {type_lower}", f"number of {type_lower}", f"how many {type_lower}s", f"{type_lower}s in the video", f"{type_lower} in the video"]):
            return f"Unique {vtype}(s) detected: {count}."

    if "most common" in question or "most frequent" in question:
        if vehicle_counts:
            most_common = max(vehicle_counts.items(), key=lambda x: x[1])
            return f"Most common vehicle: {most_common[0]} ({most_common[1]} instances)."
        return "No vehicles detected."

    if "what types" in question or "vehicle types" in question or "which vehicles" in question:
        if vehicle_counts:
            types = list(vehicle_counts.keys())
            types_str = ", ".join(types[:-1]) + " and " + types[-1] if len(types) > 1 else types[0]
            return f"Vehicle types: {types_str}."
        return "No vehicles detected."

    if "traffic" in question and ("condition" in question or "overall" in question):
        total = sum(vehicle_counts.values())
        condition = "heavy" if total > 30 else "moderate" if total > 15 else "light"
        return f"Traffic was {condition} with {total} unique vehicles."

    if "congestion" in question or "congested" in question:
        avg_congestion = np.mean(congestion_indices) if congestion_indices else 0
        level = "high" if avg_congestion > 2 else "moderate" if avg_congestion > 1 else "low"
        return f"Average congestion index: {avg_congestion:.2f} ({level})."

    if "emergency" in question or "alerts" in question or "ambulance activity" in question:
        if emergency_alerts:
            return f"Emergency alerts: {'; '.join(emergency_alerts)}."
        return "No emergency alerts detected."

    # Rule-based fallback if chatbot is unavailable
    if not chatbot:
        if any(kw in question for kw in ["describe", "tell me", "what can you say", "interesting", "summarize", "activity", "happening"]):
            total = sum(vehicle_counts.values())
            condition = "heavy" if total > 30 else "moderate" if total > 15 else "light"
            return f"The video shows {total} vehicles ({', '.join([f'{v}: {c}' for v, c in vehicle_counts.items()])}), with {condition} traffic, peaking at {max_vehicles} vehicles at {max_time_sec:.2f}s."
        if any(kw in question for kw in ["would", "might", "could", "cause"]):
            return f"Based on the data, a peak of {max_vehicles} vehicles at {max_time_sec:.2f}s could be due to a traffic signal, intersection delay, or temporary obstruction."
        return f"No chatbot available. Summary: {', '.join([f'{v}: {c}' for v, c in vehicle_counts.items()])}."

    # Chatbot for open-ended and hypothetical questions
    try:
        is_exploratory = any(kw in question for kw in ["describe", "tell me", "what can you say", "interesting", "summarize", "activity", "happening"])
        is_hypothetical = any(kw in question for kw in ["would", "might", "could", "cause"])

        if is_exploratory:
            prompt = (
                f"{context}\n"
                f"Instruction: Summarize the traffic activity concisely, mentioning vehicle types, counts, speeds, and trends. Start with 'The video shows...'.\n"
                f"Question: {question}\nAnswer:"
            )
        elif is_hypothetical:
            prompt = (
                f"{context}\n"
                f"Instruction: Speculate on the traffic data, discussing possible causes or impacts. Start with 'Based on the data...'.\n"
                f"Question: {question}\nAnswer:"
            )
        else:
            prompt = (
                f"{context}\n"
                f"Instruction: Answer briefly using the data, focusing on counts, speeds, or congestion.\n"
                f"Question: {question}\nAnswer:"
            )

        response = chatbot(prompt, max_length=100, num_return_sequences=1)[0]['generated_text'].strip()
        if response and len(response) > 5:
            sentences = re.split(r'(?<=[.!?])\s+', response)
            return " ".join(sentences[:min(2, len(sentences))]).strip()
        else:
            return "I couldn't generate a response. Please try rephrasing your question."
    except Exception as e:
        print(f"Chatbot error: {e}")
        return f"Error processing question. Summary: {', '.join([f'{v}: {c}' for v, c in vehicle_counts.items()])}."

# Dynamic chat suggestions
suggestions = [
    "How many cars were detected in total?",
    "How many ambulances were in the video?",
    "How many vehicles at 5 seconds?",
    "What was the peak traffic time?",
    "What types of vehicles were detected?",
    "Tell me about the vehicles in the video.",
    "What can you say about the traffic flow?",
    "What was happening in the middle of the video?",
    "Was there anything interesting in the video?",
    "What might cause a peak like this?",
    "How could this data help traffic management?",
    "What was the average speed of cars?",
    "How congested was the traffic?"
]
if emergency_alerts:
    suggestions.append("When were ambulances most active?")

# Chat loop with guidance
print("\nStarting chat interaction...")
print("Video processing complete. Ask any question about the video (type 'exit' to quit).")
print("Suggested questions:")
for i, suggestion in enumerate(suggestions, 1):
    print(f"{i}. {suggestion}")
while True:
    try:
        user_query = input("Ask a question: ")
        if user_query.lower().strip() == "exit":
            print("Goodbye.")
            break
        response = answer_question(user_query, vehicle_counts, frame_vehicle_counts, fps, total_frames, average_counts, max_time_sec, max_vehicles, context, chatbot, average_speeds, congestion_indices, emergency_alerts)
        print("\nAnswer:")
        print(response)
        print()
    except KeyboardInterrupt:
        print("\nChat interrupted. Type 'exit' to quit or continue asking questions.")
    except Exception as e:
        print(f"Error in chat loop: {e}")
        print("Please try again or type 'exit' to quit.")

Installing dependencies...
YOLO model loaded successfully.
Processing video...

0: 480x640 1 Bus, 8 Cars, 99.2ms
Speed: 3.3ms preprocess, 99.2ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Bus, 8 Cars, 94.7ms
Speed: 2.4ms preprocess, 94.7ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Bus, 8 Cars, 85.5ms
Speed: 2.3ms preprocess, 85.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Bus, 8 Cars, 82.6ms
Speed: 2.4ms preprocess, 82.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Bus, 8 Cars, 82.5ms
Speed: 2.7ms preprocess, 82.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Bus, 8 Cars, 86.5ms
Speed: 2.3ms preprocess, 86.5ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Bus, 8 Cars, 85.8ms
Speed: 2.6ms preprocess, 85.8ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 

Device set to use cpu


Flan-T5-Small loaded for report generation.
Loading Flan-T5-Large chatbot model...


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


Flan-T5-Large loaded for chatbot.
Generating traffic density heatmap...
Heatmap saved as 'heatmap.png'.
Exporting data to CSV...
Traffic data exported to 'traffic_data.csv'.
Generating text report...
Text report generated with text model.

--- Generated Report ---

Vehicle Detection Report: Vehicles are grouped into two frames: Car and Bus.
Generating visualization dashboard...
Dashboard saved as 'dashboard.png'.
Generating PDF report...
PDF report generated as 'report.pdf'.

Chatbot context:
 Video Analysis: 301 frames, 10.03s, 30.00 FPS.
Unique vehicles: Car: 18, Bus: 1 (total: 19).
Average per frame: Car: 3.11, Bus: 0.02.
Average speeds (pixels/s): Car: 37.69, Bus: 15.82.
Peak traffic: 5 vehicles at 1.33s.
Middle frame (5.00s): Car: 4.
Vehicle types: Car, Bus.
Traffic: Moderate.
Congestion index: 0.63 (0=low, 1=moderate, >2=high).


Starting chat interaction...
Video processing complete. Ask any question about the video (type 'exit' to quit).
Suggested questions:
1. How many cars we